In [15]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
import mlflow
import mlflow.spark
import pickle

Basicamente aqui vou fazer a criação do  mdelos com as features escolhida dentro do EAD  modelo este que tem a intenção de prever o preço peixe com base me certas carcteristicas

In [16]:
spark = SparkSession.builder \
    .appName("FishRegressionFinal") \
    .config("spark.ui.showConsoleProgress", "false") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()
mlflow.set_experiment("Regressao_Peixes")

spark

In [17]:
df = spark.read.parquet("fish_processed.parquet", header=True, inferSchema=True)
df.show(5)


+---------------+-----+---------+--------+-----------------------+-----------+------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-------+-------+----+-----------+--------------+
|           Name| Sell|where_row|  Shadow|Total Catches to Unlock|Spawn Rates|Rain/Snow Catch Up|     NH Jan|     NH Feb|     NH Mar|     NH Apr|     NH May|     NH Jun|     NH Jul|     NH Aug|     NH Sep|     NH Oct|     NH Nov|     NH Dec|     SH Jan|     SH Feb|     SH Mar|     SH Apr|     SH May|     SH Jun|     SH Jul|     SH Aug|     SH Sep|     SH Oct|     SH Nov|     SH Dec|Color 1|Color 2|Size| light_type|Spawn_Rate_Avg|
+---------------+-----+---------+--------+-----------------------+-----------+------------------+-----------+-----------+-----------

In [18]:
categorical_cols = [col for col, dtype in df.dtypes if dtype == "string"]
df = df.fillna("Unknown", subset=categorical_cols)

In [19]:
train_raw, test_raw = df.randomSplit([0.8, 0.2], seed=42)
print(f"\nDivisão: Treino={train_raw.count()}, Teste={test_raw.count()}")


Divisão: Treino=63, Teste=17


In [20]:


cat_features = [ "where_row", "Shadow","light_type"]
num_features = ["Total Catches to Unlock", "Spawn_Rate_Avg"]


indexers = [
    StringIndexer(inputCol=col, outputCol=col + "_indexed", handleInvalid="skip")
    for col in cat_features
]


encoders = [
    OneHotEncoder(inputCol=col + "_indexed", outputCol=col + "_vec")
    for col in cat_features
]


final_features = num_features + [col + "_vec" for col in cat_features]

print("As features finais para o modelo serão:")
print(final_features)


assembler = VectorAssembler(inputCols=final_features, outputCol="features")


pipeline = Pipeline(stages=indexers + encoders + [assembler])


pipeline_model = pipeline.fit(df)
df = pipeline_model.transform(df).select("features", "Sell")

print("\nPré-processamento ML concluído. Esquema final:")
df.printSchema()


train, test = df.randomSplit([0.8, 0.2], seed=42)
print(f"\nDivisão de dados: Treino={train.count()}, Teste={test.count()}")


As features finais para o modelo serão:
['Total Catches to Unlock', 'Spawn_Rate_Avg', 'where_row_vec', 'Shadow_vec', 'light_type_vec']

Pré-processamento ML concluído. Esquema final:
root
 |-- features: vector (nullable = true)
 |-- Sell: long (nullable = true)


Divisão de dados: Treino=63, Teste=17


In [21]:
odels = {
    "LR": LinearRegression(labelCol="Sell", featuresCol="features"),
    "RF": RandomForestRegressor(labelCol="Sell", featuresCol="features", seed=42),
    "GBT": GBTRegressor(labelCol="Sell", featuresCol="features", seed=42)
}

param_grids = {
    "LR": ParamGridBuilder()
        .addGrid(models["LR"].regParam, [0.0, 0.1, 0.2])
        .addGrid(models["LR"].elasticNetParam, [0.0, 0.5, 1.0])
        .build(),
    
    "RF": ParamGridBuilder()
        .addGrid(models["RF"].numTrees, [50, 100])
        .addGrid(models["RF"].maxDepth, [5, 10])
        .build(),
    
    
    "GBT": ParamGridBuilder()
        .addGrid(models["GBT"].maxDepth, [3, 5])
        .addGrid(models["GBT"].maxIter, [20, 30])  
        .build()
}



In [22]:
evaluator = RegressionEvaluator(
    labelCol="Sell", 
    predictionCol="prediction", 
    metricName="rmse"
)

results = {}

for model_name in ["LR", "RF", "GBT"]:
    with mlflow.start_run(run_name=model_name):
        print(f"\n{'='*60}")
        print(f"Treinando: {model_name}")
        print(f"{'='*60}")
        
       
        cv = CrossValidator(
            estimator=models[model_name],
            estimatorParamMaps=param_grids[model_name],
            evaluator=evaluator,
            numFolds=5
        )
        
        
        cv_model = cv.fit(train)
        
       
        train_pred = cv_model.transform(train)
        test_pred = cv_model.transform(test)
        
        
        train_rmse = evaluator.evaluate(train_pred)
        test_rmse = evaluator.evaluate(test_pred)
        
       
        r2_evaluator = RegressionEvaluator(
            labelCol="Sell", 
            predictionCol="prediction", 
            metricName="r2"
        )
        train_r2 = r2_evaluator.evaluate(train_pred)
        test_r2 = r2_evaluator.evaluate(test_pred)
        
        results[model_name] = {
            "model": cv_model,
            "train_rmse": train_rmse,
            "test_rmse": test_rmse,
            "train_r2": train_r2,
            "test_r2": test_r2
        }
        
       
        mlflow.log_metric("train_rmse", train_rmse)
        mlflow.log_metric("test_rmse", test_rmse)
        mlflow.log_metric("train_r2", train_r2)
        mlflow.log_metric("test_r2", test_r2)
        mlflow.log_param("num_folds", 5)
        
        print(f"Train RMSE: {train_rmse:.4f} | Train R²: {train_r2:.4f}")
        print(f"Test RMSE:  {test_rmse:.4f} | Test R²:  {test_r2:.4f}")



Treinando: LR
Train RMSE: 1052.3688 | Train R²: 0.9404
Test RMSE:  1733.1688 | Test R²:  0.8816

Treinando: RF
Train RMSE: 819.4901 | Train R²: 0.9638
Test RMSE:  2000.3942 | Test R²:  0.8423

Treinando: GBT
Train RMSE: 455.1168 | Train R²: 0.9888
Test RMSE:  2070.5739 | Test R²:  0.8310


In [23]:
best_model_name = min(results, key=lambda k: results[k]["test_rmse"])
best_model = results[best_model_name]["model"].bestModel

print(f"Melhor modelo: {best_model_name}")

Melhor modelo: LR


In [26]:
import os

# Caminho da instalação do Hadoop
os.environ["HADOOP_HOME"] = r"C:\hadoop\hadoop-3.3.6"
os.environ["PATH"] += os.pathsep + r"C:\hadoop\hadoop-3.3.6\bin"


In [31]:
from pyspark.ml.pipeline import PipelineModel


best_model_name = min(results, key=lambda k: results[k]["test_rmse"])
best_model = results[best_model_name]["model"].bestModel


save_path = r"C:\Animal-crossing-performace\models\linear_regression"

best_model.write().overwrite().save(save_path)

print(f"Modelo salvo em: {save_path}")


Py4JJavaError: An error occurred while calling o121177.save.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:314)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:1116)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:798)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:838)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:810)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:837)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdir

o salvamento do spark não esta fucionado que legal